# Getting Started

This is a short example showing how to download blockchain records for the known Quadriga cold wallets and cross reference them with ShapeShift.

You'll need an etherscan api key

https://etherscan.io/apis

A mongo DB instance and set variables in SETTINGS

Run the pip from the blockchain-scraper directory

```pip install -r requirements.txt``` 

Data collected here was in part used for [this artcle](https://www.wsj.com/articles/our-cash-went-to-something-customers-hunt-for-bankrupt-crypto-exchanges-missing-millions-11550596908)


### TODO

Still a few cleanup items I wasn't able to address before committing this which I will address based on interest

merge rawaddr_illicit_spider and blockchaininfo spider
* they are the same but one tags with a search addr which should be an optional parameter

add indices for the block crawler, wallet_explorer

write some documentation to set up

In [1]:
import pymongo
from pymongo import MongoClient
from bson.objectid import ObjectId
from scrapy.conf import settings

import pandas as pd
import requests
from bs4 import BeautifulSoup
from datetime import datetime

connection = MongoClient(
    settings['MONGODB_SERVER'],
    settings['MONGODB_PORT']
)
ss_db = connection[settings['SHAPESHIFT_DB']]
eth_db = connection[settings['ETH_DB']]
btc_db = connection[settings['BTC_DB']]
api_token = settings['ETHERSCAN_KEY']

/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: ScrapyDeprecationWarning: Module `scrapy.conf` is deprecated, use `crawler.settings` attribute instead
  after removing the cwd from sys.path.


### Create a ETH/USD lookup

In [66]:
with open('data/eth-prices.csv', 'wb') as infile:
    infile.write(requests.get('https://etherscan.io/chart/etherprice?output=csv').content)

eth_conversions = pd.read_csv('data/eth-prices.csv')
eth_conversions['dt'] = eth_conversions['UnixTimeStamp'].apply(lambda x: datetime.fromtimestamp(x))
eth_conversions['dt_str'] = eth_conversions['dt'].apply(lambda x: x.strftime('%m%d%y'))

eth_conversions_lookup = {}
for row in eth_conversions.iterrows():
    obj = row[1]
    eth_conversions_lookup[obj['dt_str']] = obj['Value']

eth_conversions.head()

,Date(UTC),UnixTimeStamp,Value,dt,dt_str
0,7/30/2015,1438214400,0.0,2015-07-30,073015
1,7/31/2015,1438300800,0.0,2015-07-31,073115
2,8/1/2015,1438387200,0.0,2015-08-01,080115
3,8/2/2015,1438473600,0.0,2015-08-02,080215
4,8/3/2015,1438560000,0.0,2015-08-03,080315


### Get some initial labels for addresses

In [3]:
exchange_addrs = pd.read_csv('https://docs.google.com/spreadsheets/d/e/2PACX-1vQL2mYYqRA2-UnDr22xWrwsENjV6n5DtMXS9sEceUKN6dF-IzAtJsve8TPXPlXgioQ5MMac3RbmuL8m/pub?gid=0&single=true&output=csv')
exchange_addrs['addr'] = exchange_addrs['addr'].apply(lambda x: x.lower())

exchange_lookup = {}
for cx in exchange_addrs.iterrows():
    obj = cx[1]
    if obj['name'] in exchange_lookup.keys():
        exchange_lookup[obj['name']].append(obj['addr'])
    else:
        exchange_lookup[obj['name']] = [obj['addr']]

        
ss_addrs = list(set(exchange_addrs[exchange_addrs.name.str.contains('shapeshift')].addr))
print(len(ss_addrs))
exchange_addrs.head(10)

8


,addr,name
0,0x70faa28a6b8d6829a4b1e649d26ec9a2a39ba413,shapeshift
1,0x120a270bbc009644e35f0bb6ab13f95b8199c4ad,shapeshift
2,0x9e6316f44baeeee5d41a1070516cc5fa47baf227,shapeshift
3,0x3b0bc51ab9de1e5b7b6e34e5b960285805c41736,shapeshift
4,0x563b377a956c80d77a7c613a9343699ad6123911,shapeshift
5,0xd3273eba07248020bf98a8b560ec1576a612102f,shapeshift
6,0xeed16856d551569d134530ee3967ec79995e2051,shapeshift
7,0x1c39ba39e4735cb65978d4db400ddd70a72dc750,shapeshift contract
8,0xc3cae4118fec40ef386e01eb04b7e66dc0e5b643,quadriga cold wallet
9,0x7ea5e875a386b66d11a0ad1866ca7b5f2745f049,quadriga cold wallet


### Create indices

Set up indexes that will prevent duplicate entries and make lookups fast. Probably should only be run once.

In [4]:
create_indices = False
print(eth_db.etherscan_url.count_documents({}))
if create_indices:
    eth_db.etherscan_url.delete_many({})
    eth_db.etherscan_url.drop_indexes()
    eth_db.etherscan_url.create_index([('url', pymongo.DESCENDING)], unique=True)
    eth_db.etherscan_url.create_index([('retrieved', pymongo.DESCENDING)], unique=False)
    eth_db.etherscan_url.create_index([('last_scraped', pymongo.DESCENDING)], unique=False)
    
    eth_db.etherscan_tx.drop_indexes()
    eth_db.etherscan_tx.create_index([('hash', pymongo.DESCENDING)], unique=True)
    eth_db.etherscan_tx.create_index([('to', pymongo.DESCENDING)], unique=False)
    eth_db.etherscan_tx.create_index([('from', pymongo.DESCENDING)], unique=False)
    eth_db.etherscan_tx.create_index([('eth', pymongo.DESCENDING)], unique=False)
    eth_db.etherscan_tx.create_index([('dt', pymongo.DESCENDING)], unique=False)
    
print(btc_db.blockchaininfo_url.count_documents({}))
if create_indices:
    btc_db.blockchaininfo_url.delete_many({})
    btc_db.blockchaininfo_url.drop_indexes()
    btc_db.blockchaininfo_url.create_index([('url', pymongo.DESCENDING)], unique=True)
    btc_db.blockchaininfo_url.create_index([('retrieved', pymongo.DESCENDING)], unique=False)
    btc_db.blockchaininfo_url.create_index([('last_scraped', pymongo.DESCENDING)], unique=False)
    
    btc_db.blockchaininfo_tx.drop_indexes()
    btc_db.blockchaininfo_tx.create_index([
        ('hash', pymongo.DESCENDING),
    ],  unique=True)
    btc_db.blockchaininfo_tx.create_index([
        ('dt', pymongo.DESCENDING)
    ])
    
    btc_db.blockchaininfo_wallet_tx.drop_indexes()
    btc_db.blockchaininfo_wallet_tx.create_index([
        ('hash', pymongo.DESCENDING),
        ('addr', pymongo.DESCENDING),
        ('dir', pymongo.DESCENDING)
    ],  unique=True)
    btc_db.blockchaininfo_wallet_tx.create_index([
        ('addr', pymongo.DESCENDING)
    ])
    btc_db.blockchaininfo_wallet_tx.create_index([
        ('dt', pymongo.DESCENDING)
    ])
    btc_db.blockchaininfo_wallet_tx.create_index([
        ('btc', pymongo.DESCENDING)
    ])
    btc_db.blockchaininfo_wallet_tx.create_index([
        ('dir', pymongo.DESCENDING)
    ])
    
    btc_db.wallet_summary.create_index([
        ('address', pymongo.DESCENDING)
    ],  unique=True)

    
print(btc_db.blockchaininfo_url.find({}).count())


if create_indices:
    btc_db.wallet_explorer_addr.drop_indexes()
    btc_db.wallet_explorer_addr.create_index([
        ('wallet_address', pymongo.DESCENDING),
        ('exchange_name', pymongo.DESCENDING),
    ], unique=True)
        
    btc_db.wallet_explorer_exchange.drop_indexes()
    btc_db.wallet_explorer_exchange.create_index([
        ('tx_url', pymongo.DESCENDING),
        ('wallet_url', pymongo.DESCENDING),
        ('exchange_name', pymongo.DESCENDING),
    ], unique=True)

print(ss_db.ss_url.count_documents({}))
print(ss_db.ss_resp.count_documents({}))

if create_indices:
    #ss_db.ss_url.delete_many({})
    ss_db.ss_url.drop_indexes()
    ss_db.ss_url.create_index([('url', pymongo.DESCENDING)], unique=True)
    ss_db.ss_url.create_index([('retrieved', pymongo.DESCENDING)], unique=False)
    ss_db.ss_url.create_index([('last_scraped', pymongo.DESCENDING)], unique=False)
    ss_db.ss_url.create_index([('addr', pymongo.DESCENDING)], unique=False)
    ss_db.ss_url.create_index([('currency', pymongo.DESCENDING)], unique=False)
    
    ss_db.ss_resp.drop_indexes()
    ss_db.ss_resp.create_index([
        ('url', pymongo.DESCENDING),
        ('retrieved', pymongo.DESCENDING),
    ], unique=True)
    ss_db.ss_resp.create_index([('addr', pymongo.DESCENDING)], unique=False)
    ss_db.ss_resp.create_index([('currency', pymongo.DESCENDING)], unique=False)
    ss_db.ss_resp.create_index([('withdraw', pymongo.DESCENDING)], unique=False)
    ss_db.ss_resp.create_index([('outgoingType', pymongo.DESCENDING)], unique=False)

    
    
print(ss_db.ss_url.count_documents({}))
print(ss_db.ss_resp.count_documents({}))


2885
0
0
12540
2664
12540
2664


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:53: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.


### Set up URLs to scrape

We need to collect transactions associated with Quadriga's known Ethereum wallets

Using this information, create a list of URLs to request from etherscan

In [8]:
quadriga_addrs = exchange_addrs[exchange_addrs.name.str.contains('quadriga')]
quadriga_cold_addrs = list(quadriga_addrs[quadriga_addrs.name.str.contains('cold wallet')].addr)
quadriga_hot_addrs = list(quadriga_addrs[quadriga_addrs.name.str.contains('hot wallet')].addr)

print(len(quadriga_cold_addrs))
print(len(quadriga_hot_addrs))

quadriga_addrs

4
2


,addr,name
8,0xc3cae4118fec40ef386e01eb04b7e66dc0e5b643,quadriga cold wallet
9,0x7ea5e875a386b66d11a0ad1866ca7b5f2745f049,quadriga cold wallet
10,0x0b9defea64d1808bcdec76d532984dd24fb8bcff,quadriga cold wallet
11,0x0ee4e2d09aec35bdf08083b649033ac0a41aa75e,quadriga cold wallet
12,0x027beefcbad782faf69fad12dee97ed894c68549,quadriga hot wallet
13,0xb6aac3b56ff818496b747ea57fcbe42a9aae6218,quadriga hot wallet


In [10]:
#create another version o
def add_lookup(addrs, tag, page=0, offset=0):
    url = 'http://api.etherscan.io/api?module=account&action=txlist&address=%s&startblock=0&endblock=99999999&sort=asc&apikey=%s&page=%s&offset=%s'
    for addr in addrs:
        tmp_url = url % (addr, api_token, page, offset)
        obj = {
            'url': tmp_url,
            'retrieved': False,
            'last_scraped': None,
            'has_result': False,
            'tag': tag,
            'account_addr': addr
        }
        try:
            eth_db.etherscan_url.insert_many([obj])
        except:
            pass
    

In [25]:
add_lookup(quadriga_cold_addrs, 'quadriga_cold_wallet')
print('total', eth_db.etherscan_url.count_documents({}))
print('retrieved', eth_db.etherscan_url.count_documents({'retrieved': True}))
print('missing', eth_db.etherscan_url.count_documents({'retrieved': False}))
eth_db.etherscan_url.find_one({})
#we need to pay attention to this SS contract which handled a number of transactions
#it is considered an intermediary... 
#add_lookup(['0x1c39ba39e4735cb65978d4db400ddd70a72dc750'], 'ss-contract')

total 4
retrieved 0
missing 4


{'_id': ObjectId('5c7d4c005a13494cb79d5a19'),
 'url': 'http://api.etherscan.io/api?module=account&action=txlist&address=0xc3cae4118fec40ef386e01eb04b7e66dc0e5b643&startblock=0&endblock=99999999&sort=asc&apikey=WSY7HG9SNHSVYW4ZFGIIM4VBQB7ZH2235M&page=0&offset=0',
 'retrieved': False,
 'last_scraped': None,
 'has_result': False,
 'tag': 'quadriga_cold_wallet',
 'account_addr': '0xc3cae4118fec40ef386e01eb04b7e66dc0e5b643'}

# Now run the scrapy etherscan crawler

From the blockchain-scraper directory

```scrapy crawl etherscan```

In [27]:
print('total', eth_db.etherscan_url.count_documents({}))
print('retrieved', eth_db.etherscan_url.count_documents({'retrieved': True}))
print('missing', eth_db.etherscan_url.count_documents({'retrieved': False}))

total 4
retrieved 4
missing 0


In [30]:
#How many TX's went ot ShapeShift Addresses?
eth_db.etherscan_tx.count_documents({'to': {'$in': ss_addrs}})

0

At this point there should be no transactions sending to a ShapeShift we have an address for because ShapeShift deposits go to a distinct wallet. These wallets are later "swept" into the wallet addresses we defined above.

To find those addresses we need to download all transactions the cold wallets sent money to examine where the funds ended up

In [33]:
to_search = eth_db.etherscan_tx.find({'from': {'$in': quadriga_cold_addrs}}).distinct('to')
print('number of wallets to download', len(to_search))
add_lookup(to_search, 'quadriga_cold_wallet_first_hop')

number of wallets to download 3220


From the blockchain-scraper directory

```scrapy crawl etherscan```

In [34]:
print('total', eth_db.etherscan_url.count_documents({}))
print('retrieved', eth_db.etherscan_url.count_documents({'retrieved': True}))
print('missing', eth_db.etherscan_url.count_documents({'retrieved': False}))

total 3222
retrieved 4
missing 3218


In [38]:
#Now we ought to have collected all the intermediary addresses that sweep funds into the main ShapeShift wallets
eth_db.etherscan_tx.count_documents({'to': {'$in': ss_addrs}})

119

## Analyzing the Quadriga transactions involving ShapeShift

In [40]:
ss_intermediaries = pd.DataFrame.from_dict(list(eth_db.etherscan_tx.find({
    'to': {'$in': ss_addrs}
})))
print('Intermediary transactions', len(ss_intermediaries))
ss_intermediary_addrs = list(set(ss_intermediaries['from']))
print('Intermediary addresses', len(ss_intermediary_addrs))
ss_intermediaries.head()

Intermediary transactions 137
Intermediary addresses 137


,_id,account_addr,blockHash,blockNumber,confirmations,contractAddress,cumulativeGasUsed,dt,dt_str,eth,...,hash,input,isError,nonce,tag,timeStamp,to,transactionIndex,txreceipt_status,value
0,5c7d50995a13494d9ed9e76d,0xad9b49336c088420832db3794dbb32923267a713,0xc2862cdea560e457cf4bd01d949954f433b6d073ce72...,1475583,5828396,,42000,2016-05-07 20:24:05,050716,215.99958,...,0xa0ea5c060bb2a27a25be8d33812103613ac9797a1e37...,0x,0,0,quadriga_cold_wallet_first_hop,1462652645,0x9e6316f44baeeee5d41a1070516cc5fa47baf227,1,,2.159996e+20
1,5c7d50965a13494d9ed9e76b,0xd56d5a492df96b0bed9e824cdf6aa159ff19aadd,0xc2862cdea560e457cf4bd01d949954f433b6d073ce72...,1475583,5828396,,168000,2016-05-07 20:24:05,050716,212.99958,...,0x866e7e470031b43b2ad354d92157a8260e6e9b2765dc...,0x,0,0,quadriga_cold_wallet_first_hop,1462652645,0x9e6316f44baeeee5d41a1070516cc5fa47baf227,7,,2.129996e+20
2,5c7d50925a13494d9ed9e769,0x75f3a12bbb62b0f7674330fb0d83150790396f18,0xc2862cdea560e457cf4bd01d949954f433b6d073ce72...,1475583,5828396,,504000,2016-05-07 20:24:05,050716,99.99958,...,0xc6b0546458d1f8d556ebddd626834cead03e600b339b...,0x,0,0,quadriga_cold_wallet_first_hop,1462652645,0x9e6316f44baeeee5d41a1070516cc5fa47baf227,23,,9.999958e+19
3,5c7d508d5a13494d9ed9e767,0x39733d7b62350eb6dd388d123e112fe2bacdc7dc,0xc2862cdea560e457cf4bd01d949954f433b6d073ce72...,1475583,5828396,,21000,2016-05-07 20:24:05,050716,213.99958,...,0x4cdaeeec8c18ee06732e1523695c35a3e068063d201e...,0x,0,0,quadriga_cold_wallet_first_hop,1462652645,0x9e6316f44baeeee5d41a1070516cc5fa47baf227,0,,2.139996e+20
4,5c7d508a5a13494d9ed9e765,0xe8b8b0515b25d71e43c4b5aeb4ceb37f0cfcf549,0xc75ac635a082a3df78a49b6e6c8535d69ae97e5a44f1...,1474950,5829029,,172182,2016-05-07 17:44:17,050716,79.99958,...,0xf58999f43706319cbce7746cb23b98c3672e5d2cbdde...,0x,0,0,quadriga_cold_wallet_first_hop,1462643057,0x9e6316f44baeeee5d41a1070516cc5fa47baf227,5,,7.999958e+19


#### and query the ShapeShift API to collect confirmations

ShapeShift should confirm that funds were receieved from these wallets and document the destination wallets

In [42]:
print(ss_db.ss_url.count_documents({'retrieved': False}))
for addr in list(set(ss_intermediaries['from'])):
    obj = {
        'addr': addr,
        'currency': 'eth',
        'url': 'https://shapeshift.io/txStat/%s' % addr,
        'retrieved': False,
        'last_scraped': None,
        'has_result': False
    }
    try:
        ss_db.ss_url.insert_many([obj])
    except:
        pass
print(ss_db.ss_url.count_documents({'retrieved': False}))

9876
9876


From the crawler from the blockchain-scraper directory

```scrapy crawl shapeshift```

In [43]:
print('SS URLs', ss_db.ss_url.find({}).count())
print('SS URLs retrieved', ss_db.ss_url.find({'retrieved': True}).count())
print('SS responses', ss_db.ss_resp.find({}).count())

SS URLs 12540
SS URLs retrieved 2664
SS responses 2664


/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  """Entry point for launching an IPython kernel.
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  
/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:3: DeprecationWarning: count is deprecated. Use Collection.count_documents instead.
  This is separate from the ipykernel package so we can avoid doing imports until


In [44]:
ss_responses = pd.DataFrame.from_dict(list(ss_db.ss_resp.find({})))
ss_responses.head()

,_id,addr,address,incomingCoin,incomingType,outgoingCoin,outgoingType,retrieved,status,transaction,transactionURL,withdraw
0,5c66e9025a13490cf9559dea,0x7ca5d5c610c32945ef575773f559cb1d6a418185,0x7ca5d5c610c32945ef575773f559cb1d6a418185,8.6,ETH,0.64855376,BTC,2019-02-15 16:29:54.073,complete,d0be39e1c51da397e5f6d9e7f64c99ee1299f9b838400d...,https://blockchain.info/tx/d0be39e1c51da397e5f...,1D9QoGJCsorTLnCzC4dCRDVaX2WaHafLHU
1,5c66e9025a13490cf9559deb,0xc1b17e9693a78bdb5895f2d98ed8215f147024f7,0xc1b17e9693a78bdb5895f2d98ed8215f147024f7,25.0,ETH,1.73419775,BTC,2019-02-15 16:29:54.077,complete,32723612aabf9739f7bd289d5d9881ad524061079fe792...,https://blockchain.info/tx/32723612aabf9739f7b...,1KE9HQxu2ncURbELGSLP4fCT9B2wxjUnV4
2,5c66e9085a13490cf9559dec,0x5c707a0ad35e500fed97d938f3195969b2f7b342,0x5c707a0ad35e500fed97d938f3195969b2f7b342,113.0,ETH,4.12547021,BTC,2019-02-15 16:30:00.307,complete,45d02ad9c976f7ce7af27f58f39bb9eaa7f8b19da95a35...,https://blockchain.info/tx/45d02ad9c976f7ce7af...,3MBpUD3nhxJGQ6JzXBMKXvMoF82c7CVh8J
3,5c66e9095a13490cf9559ded,0x7a258e84d231677f82a9075a720a2d3209f8016b,0x7a258e84d231677f82a9075a720a2d3209f8016b,26.0,ETH,1.76673164,BTC,2019-02-15 16:30:01.169,complete,c6abcee1ec99fe24a04ae939658858dc497c0c8eb73924...,https://blockchain.info/tx/c6abcee1ec99fe24a04...,16SSLMaDqP15GnP3tDCEchqN34hSzAtGqB
4,5c66e90b5a13490cf9559dee,0xced07f6073ab56e0b8945b5da6447127f7ce1b15,0xced07f6073ab56e0b8945b5da6447127f7ce1b15,31.0,ETH,2.14354658,BTC,2019-02-15 16:30:03.880,complete,3e49f4ef534f6ad9b6930a0899b7febc57dd675f659d52...,https://blockchain.info/tx/3e49f4ef534f6ad9b69...,14T7eV8mj12EtXaURzjbV5JnvZMAZGxjez


## Hot wallets and pagination

hot wallets have numerous transactions and need to paginated to retrieve all txs

Etherscan only returns the 10,000 most-recent transactions. We need full history of Quadriga wallets so scrape the number of transactions from the Etherscan page (bc this is bizarrely not an API call I can identify) and create paginated URLs

In [ ]:
add_lookup(quadriga_hot_addrs, 'quadriga_hot_wallet')

In [17]:
qx_hot_1 = quadriga_hot_addrs[0]
resp = requests.get('https://etherscan.io/address/%s' % qx_hot_1)
soup = BeautifulSoup(resp.content, 'html.parser')
link = soup.find('div', {"id": "transactions"}).find('a')
num_tx = 0
offset = 10000
if link is not None:
    num_tx = int(link.text.split(' ')[0].replace(',', ''))

pages = round(num_tx / offset) + 1
print('number of transactions', num_tx, 'number of pages', pages)
for idx in range(1, pages):
    add_lookup([qx_hot_1], 'quadriga hotwallet page %s' % idx, idx, offset)

number of transactions 20126 number of pages 3


In [18]:
qx_hot_2 = quadriga_hot_addrs[1]
resp = requests.get('https://etherscan.io/address/%s' % qx_hot_2)
soup = BeautifulSoup(resp.content, 'html.parser')
link = soup.find('div', {"id": "transactions"}).find('a')
num_tx = 0
offset = 10000
if link is not None:
    num_tx = int(link.text.split(' ')[0].replace(',', ''))
pages = round(num_tx / offset) + 1
print('number of transactions', num_tx, 'number of pages', pages)
for idx in range(1, pages):
    add_lookup([qx_hot_2], 'quadriga hotwallet page %s' % idx, idx, offset)

number of transactions 448260 number of pages 46


### Looking at Quadriga deposits and withdrawls

In [63]:
print('num quadriga withdrawls:', eth_db.etherscan_tx.count_documents({'from': {'$in': quadriga_cold_addrs}}))

num quadriga withdrawls: 6100


In [64]:
print('num quadriga deposits:', eth_db.etherscan_tx.count_documents({'to': {'$in': quadriga_cold_addrs}}))

num quadriga deposits: 756


In [65]:
quad_cold_tx = pd.DataFrame.from_dict(list(eth_db.etherscan_tx.find({
    '$or': [{'to': {'$in': quadriga_cold_addrs}}, {'from': {'$in': quadriga_cold_addrs}}],
}, {'eth': True, 'dt': True, 'dt_str': True, 'to': True, 'from': True, 'hash': True})))
print(len(quad_cold_tx))
quad_cold_tx = quad_cold_tx.drop_duplicates(subset=['hash'])
print(len(quad_cold_tx))
#quad_cold_tx['eth'] = quad_cold_tx.apply(lambda x: 0 if (x['from'] in quadriga_cold_addrs and x['to'] in quadriga_cold_addrs) else x['eth'], axis=1)
quad_cold_tx['eth'] = quad_cold_tx.apply(lambda x: -1 * x['eth'] if x['from'] in quadriga_cold_addrs and x['to'] not in quadriga_cold_addrs else x['eth'], axis=1)
quad_cold_tx['usd'] = quad_cold_tx.apply(lambda x: float(x['eth']) * eth_conversions_lookup[x['dt_str']], axis=1)
quad_cold_tx.head()

6850
6850


,_id,dt,dt_str,eth,from,hash,to,usd
0,5c7d4c175a13494d54d3b906,2018-11-01 15:40:35,110118,140.000000,0xb6aac3b56ff818496b747ea57fcbe42a9aae6218,0x1f583509002deff2e825058caaeda7482565493196ef...,0xc3cae4118fec40ef386e01eb04b7e66dc0e5b643,27822.200000
1,5c7d4c175a13494d54d3b8f0,2018-11-01 12:29:59,110118,600.000000,0xb6aac3b56ff818496b747ea57fcbe42a9aae6218,0x88ed42d8fbc67cfec54985564968eeb499274d1b3d74...,0xc3cae4118fec40ef386e01eb04b7e66dc0e5b643,119238.000000
2,5c7d4c175a13494d54d3b8ed,2018-10-31 16:21:25,103118,25.091256,0xeed16856d551569d134530ee3967ec79995e2051,0x2ad042d7c21ed6307ef1be1f1f445c01506e8e381a59...,0xc3cae4118fec40ef386e01eb04b7e66dc0e5b643,4964.305000
3,5c7d4c175a13494d54d3b8ec,2018-10-31 16:21:08,103118,25.091256,0xd3273eba07248020bf98a8b560ec1576a612102f,0x187c8d93ad2b2d75fcfaed677671f5e2c2f9ee2a961d...,0xc3cae4118fec40ef386e01eb04b7e66dc0e5b643,4964.305000
4,5c7d4c175a13494d54d3b8ea,2018-10-31 16:14:10,103118,0.498529,0x563b377a956c80d77a7c613a9343699ad6123911,0xed04ccfcbfdad82f5b24553218d8a2c4951ab565aa39...,0xc3cae4118fec40ef386e01eb04b7e66dc0e5b643,98.633931
